# Stream & Chill: Finding the Best Date Night 'Background' Movie 

This notebook pulls movie detail information using the **TMDB.com API** found at (https://www.themoviedb.org/). Using that info, it generates a score for each movie based on how well that movie will fair as a 'background' movie to watch on a date. Using information provided by JustWatch, we can also figure out what streaming services these movies are available on. All this information on the movies and streaming availability is saved onto dataframes and connected to a Postgres database.

This is broken into six parts:
> 1. Create movies dataframe
2. Generate a 'sexiness score' for each movie
3. Create streaming availability dataframe
4. Create a genre dataframe
5. Clean the movie info dataframe
6. Connect and send dataframes to postgres database


**Limitations:** TMDB API limits the number of movies you can pull information for and requires an individual API call to get details for each movie.

In [1]:
#import libraries
import pandas as pd
import json
import requests
import concurrent.futures as cf
import psycopg2
from sqlalchemy import create_engine

## 1. Create Movies Dataframe:

In [2]:
#set number of movies you want to pull
num_movies = 10000 # min 20; max 10,000; must be multiples of 20
num_pages = int(round(num_movies/20,0))

In [3]:
#tmdb api authkey
auth_key = ""

In [4]:
#create list of top most popular movies
movie_ids = []
def get_movie_ids(page_number):

    url = "https://api.themoviedb.org/3/movie/popular?language=en-US&page=" + str(page_number)
    headers = {
        "accept": "application/json",
        "Authorization": auth_key
    }

    popular_movies = requests.get(url, headers=headers).json()['results']
    
    for i in range(len(popular_movies)):
        movie_ids.append(popular_movies[i]['id'])

In [5]:
%%time
#create a list of the movie id's 
#makes multiple API calls concurrently 
with cf.ThreadPoolExecutor() as executor:
    executor.map(get_movie_ids, range(1,num_pages+1))
movie_ids = list(dict.fromkeys(movie_ids))
print(len(movie_ids), "movies successfully stored in list!")

8305 movies successfully stored in list!
CPU times: user 8.19 s, sys: 1.57 s, total: 9.77 s
Wall time: 9.44 s


In [6]:
#get a movie details function
popular_movies_list = []
def get_movie_details(movie_id):
    url = "https://api.themoviedb.org/3/movie/" + str(movie_id) + "?append_to_response=watch%2Fproviders&language=en-US"

    headers = {
        "accept": "application/json",
        "Authorization": auth_key
    }


    popular_movies = requests.get(url, headers=headers).json()
    popular_movies_list.append(popular_movies)

In [7]:
%%time
#create a dataframe with all movies' details
with cf.ThreadPoolExecutor() as executor:
    executor.map(get_movie_details, movie_ids)

CPU times: user 2min 22s, sys: 25 s, total: 2min 47s
Wall time: 2min 41s


In [8]:
#convert list to dataframe
all_movies_df = None
all_movies_df = pd.DataFrame(popular_movies_list)
all_movies_df.rename(columns={'id':'movie_id', 'watch/providers': 'watch_providers'}, inplace= True)

## 2. Generate a 'Sexiness Score' for Each Movie:
The score on how well the movie would work for mostly 'background' noise is based on a few criteria:
 > - **10% relevancy**: Movie should be somewhat known, distracting if it is too new, which is why it has a small weight.
 > - **50% runtime**: Optimal length to leave enough time for all activities is 140 min (2hr 20min). If movie is too short it will cause users to stop their date to find another. Movie should ideally finish as users are wrapping up their date activities.
 > - **50% Boring-ness**: A really bad movie will force user to change movie. A really good movie could be distraction. Ideal score is 7/10; enough to capture attention at first, but not interesting enough to capture too much of user attention.

In [9]:
%%time
# generate sexiness score and add it as a column to the dataframe
sexiness_score_list = []
for movie_id in all_movies_df['movie_id']:
    relevancy = all_movies_df[all_movies_df['movie_id'] == movie_id]['popularity'].values[0]/10000

    runtime_score = (1 - abs(all_movies_df[all_movies_df['movie_id'] == movie_id]['runtime'].values[0]-140)/100)
    if runtime_score <= 0:
        runtime_score = 0
    vote_average = all_movies_df[all_movies_df['movie_id'] == movie_id]['vote_average'].values[0]
    if vote_average >= 7.5:
        uninteresting_score = 1 - (vote_average-7.5)/5
    else:
        uninteresting_score = pow(1.8,(vote_average - 7.5))

    sexiness_score = (0.1 * relevancy) + (0.5 * runtime_score) + (0.4 * uninteresting_score)

    sexiness_score_list.append(sexiness_score)
    
all_movies_df['sexiness_score'] = sexiness_score_list

CPU times: user 8.46 s, sys: 72.4 ms, total: 8.53 s
Wall time: 8.82 s


In [10]:
#create a top streaming providers list
streaming_provider_ids = pd.read_csv("streaming_provider_ids.csv")
top_providers_list = streaming_provider_ids['streaming_provider'].tolist()

## 3. Create Streaming Availability Dataframe:

#### Function will take the streaming availability column for a movie ID and turn it into its own dataframe:

API returns heavily nested JSON text. Relevant information is found inside of several layers:

>**Example:** to get the first listed streaming service for a movie in the U.S watch region will look as follows:
>> api_response['results']['US']['flatrate'][0]['provider_name']

_'Flatrate' means it is included in subscription service_

In [11]:
#function for getting streaming provider info per movie
def get_providers(movie_id):
    df = pd.DataFrame(columns=['movie_id','watch_region','streaming_provider'])
    provider_list = {}
    watch_info = all_movies_df[all_movies_df['movie_id'] == movie_id]['watch_providers'].tolist()[0]['results']
    for country in watch_info:
        try:
            free_availability = watch_info[country]['flatrate']
        except:
            pass
        else:
            for provider in free_availability:
                if provider['provider_name'] in top_providers_list:
                    df2 = pd.DataFrame(data = [{'movie_id': movie_id, 'watch_region': country, 'streaming_provider': provider['provider_name']}])
                    df = pd.concat([df, df2], ignore_index = True)
    return df;

In [12]:
%%time
#create streaming data frame
streaming_df = pd.DataFrame(columns=['movie_id','watch_region','streaming_provider'])
for movie_id in movie_ids:
    streaming_df = pd.concat([streaming_df, get_providers(movie_id)], ignore_index = True)
streaming_df

CPU times: user 2min 56s, sys: 4.62 s, total: 3min 1s
Wall time: 3min 9s


,movie_id,watch_region,streaming_provider
0,11036,AD,Netflix
1,11036,AD,Amazon Prime Video
2,11036,AG,Max
3,11036,AR,Max
4,11036,BA,Netflix
...,...,...,...
223612,34806,UY,Netflix
223613,34806,VE,Netflix
223614,34806,YE,Netflix
223615,34806,ZA,Netflix


## 4. Create a Genre Dataframe:

Because a movie will fit into multiple genres, a separate dataframe will better present the genres of each movie.

In [13]:
#create genre dataframe
genres_df = pd.DataFrame(columns=['movie_id','genre_id','genre_name'])
for movie_id in movie_ids:
    genre_info = all_movies_df['genres'][all_movies_df.index[all_movies_df['movie_id'] == 573435].tolist()[0]]
    for i in range(len(genre_info)):
        genre_id = genre_info[i]['id']
        genre_name = genre_info[i]['name']
        temp_df = [{'movie_id': movie_id, 'genre_id': genre_id, 'genre_name' : genre_name}]
        genres_df = genres_df._append(temp_df, ignore_index = True)
genres_df.head()

,movie_id,genre_id,genre_name
0,11036,28,Action
1,11036,80,Crime
2,11036,53,Thriller
3,11036,35,Comedy
4,104,28,Action


## 5. Clean The Movie Info Dataframe:

In [14]:
#eliminate unnecessary columns and reorder columns in all_movies_df
important_columns = [ 'movie_id','title', 'runtime', 'sexiness_score', 'original_language',  'overview', 'popularity', 'release_date', 'vote_average', 'vote_count','budget','revenue']
all_movies_df = all_movies_df.loc[:, all_movies_df.columns.isin(important_columns)]
all_movies_df = all_movies_df.reindex(columns=important_columns)
all_movies_df.head()

,movie_id,title,runtime,sexiness_score,original_language,overview,popularity,release_date,vote_average,vote_count,budget,revenue
0,104,Run Lola Run,81,0.562269,de,Lola receives a phone call from her boyfriend ...,163.296,1998-08-20,7.300,2174,1530000,7656558
1,11036,The Notebook,123,0.785437,en,An epic love story centered around an older ma...,163.727,2004-06-25,7.890,11295,29000000,115600000
2,102382,The Amazing Spider-Man 2,141,0.720029,en,"For Peter Parker, life is busy. Between taking...",162.857,2014-04-16,6.509,13008,200000000,708962323
3,566525,Shang-Chi and the Legend of the Ten Rings,132,0.857628,en,Shang-Chi must confront the past he thought he...,162.812,2021-09-01,7.550,9188,150000000,432243292
4,766507,Prey,100,0.685608,en,"When danger threatens her camp, the fierce and...",160.800,2022-08-02,7.700,6471,65000000,0


In [15]:
#change movie_id data type to object
all_movies_df['movie_id'] = all_movies_df['movie_id'].astype(object)

## 6. Connect and Send Dataframes to Postgres Database:

In [23]:
# SQL database properties
hostname = 'localhost'
database = 'StreamingDatabase'
username = 'postgres'
pwd = ''
port_id = '5433'

In [24]:
# connect to postgres database
conn = psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id)
cur = conn.cursor()

In [25]:
#function to send dfs to postgres
def df_to_sql(df, table_name):
    engine = create_engine('postgresql://' + username + ':' + pwd + '@' + hostname + ':' + port_id + '/' + database, echo = False)
    df.to_sql(name = table_name, con = engine, if_exists = 'replace', index = False)
    engine.dispose()

In [26]:
# delete dependent view
delete_view = ''' DROP VIEW IF EXISTS streaming_info_view'''
cur.execute(delete_view)
conn.commit()

In [27]:
#send the dfs to postgres database
df_to_sql(all_movies_df, 'popular_movies')
df_to_sql(streaming_df, 'streaming_availability')
df_to_sql(genres_df, 'genres')

In [28]:
#create view
create_view = '''CREATE VIEW streaming_info_view AS 
                 SELECT s.movie_id, watch_region, streaming_provider, sexiness_score, title, original_language, release_date, overview
                 FROM streaming_availability as s 
                 LEFT JOIN popular_movies as p 
                 ON s.movie_id = p.movie_id'''
cur.execute(create_view)
conn.commit()

In [29]:
# close out connections
cur.close()
conn.close()